In [32]:
# scaling the features by using the standard scaler
# remove frequencies prior to 129 Hz

# algorithm parameter:
#      100 total trees in RF
#      select 10 trees by GA
#      run 300 tests



In [33]:
# library for GA
import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

# library for RF
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


import pandas as pd
from seaborn import violinplot
import seaborn as sns

from sklearn.preprocessing import StandardScaler

# read data
spectrum_data = pd.read_excel('./spectrum_data/Nothing_Mine_CEandRock.xlsx',  sep = ',', header = 0)
# remove freqs prior to 129
spectrum_data_new = spectrum_data.iloc[:, 400:1539]


# df.insert(loc=idx, column='A', value=new_col)
spectrum_data_new.insert(loc = 0, column = 'Signal Path', value = spectrum_data.iloc[:, 0])
spectrum_data_new[0:5]

,Signal Path,Frequency_129dot8405,Frequency_130dot166,Frequency_130dot4914,Frequency_130dot8168,Frequency_131dot1422,Frequency_131dot4676,Frequency_131dot793,Frequency_132dot1185,Frequency_132dot4439,...,Frequency_497dot234,Frequency_497dot5594,Frequency_497dot8848,Frequency_498dot2102,Frequency_498dot5356,Frequency_498dot861,Frequency_499dot1865,Frequency_499dot5119,Frequency_499dot8373,Label
0,1,-82.338222,-84.856415,-84.857809,-83.830094,-85.688775,-83.544618,-83.965124,-85.252199,-84.495197,...,-86.637317,-86.343540,-86.970514,-87.039263,-88.502441,-90.199369,-87.116829,-88.726635,-90.726685,0
1,1,-88.912104,-88.837733,-89.573053,-90.704677,-87.893270,-87.805782,-86.055104,-87.536184,-92.052088,...,-92.337580,-94.363767,-92.113655,-91.933969,-91.717233,-91.703702,-94.641164,-90.733663,-89.704103,0
2,1,-84.016012,-84.131448,-88.740907,-87.258607,-86.179744,-85.351962,-85.685550,-84.290518,-84.573309,...,-90.403409,-91.020674,-88.187210,-85.616805,-86.914083,-91.192905,-91.401812,-91.905812,-86.226784,0
3,1,-90.165488,-89.575706,-81.921091,-82.703704,-85.714261,-87.114000,-86.829531,-88.225574,-87.459190,...,-90.557058,-91.756146,-89.087595,-90.093507,-89.855050,-90.758625,-88.137099,-89.833695,-89.888380,0
4,1,-87.726753,-89.734590,-87.878417,-89.838251,-90.120643,-87.962830,-89.312585,-89.720013,-87.447822,...,-91.405036,-90.592510,-91.839187,-93.906633,-91.387335,-93.815508,-93.235216,-92.409103,-93.528927,0


In [34]:
# set algorithm parameters

def run_GA(NUM_TREES, IND_SIZE, POP_SIZE, CX_RATE = 0.8):
    """
    NUM_TREES is the number of trees the random forest model pro
    """    
    MUTATE_RATE = 1.0/IND_SIZE 
    
    
    
    # prepare data: for each label, split the data into 25% test and 75 train
    raw = spectrum_data_new.values

    X_0 = raw[0:100, 0:1139]
    y_0 = raw[0:100, 1139]

    X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0, y_0, test_size=0.25, random_state=random.randint(0,5000))

    X_1 = raw[100:200, 0:1139]
    y_1 = raw[100:200, 1139]

    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.25, random_state=random.randint(0,5000))

    X_2 = raw[200:300, 0:1139]
    y_2 = raw[200:300, 1139]

    X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.25, random_state=random.randint(0,5000))

    y_test = np.concatenate((y_test_0, y_test_1, y_test_2), axis=0)
    y_train = np.concatenate((y_train_0, y_train_1, y_train_2), axis=0)
    X_test = np.concatenate((X_test_0, X_test_1, X_test_2), axis=0)
    X_train = np.concatenate((X_train_0, X_train_1, X_train_2), axis=0)
    
    # Feature Scaling
    end = 1139
    scaler = StandardScaler()
    X_train[:, 1:end] = scaler.fit_transform(X_train[:, 1:end])   # Fit to data, then transform it. Fit means Compute the mean and std to be used for later scaling.
    X_test[:, 1:end] = scaler.transform(X_test[:, 1:end]) # Perform standardization by centering and scaling
    
    model = RandomForestClassifier(n_estimators= NUM_TREES) # create a random forest with NUM_TREES = 20 
    model.fit(X_train, y_train) # train the model
    estimators = model.estimators_ # get all the trees
    
    # implement individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    # implement functions for initialize population and create individual
    toolbox = base.Toolbox() # create a toolbox of operators for our GA algorithm
    toolbox.register("indices", random.sample, range(NUM_TREES), NUM_TREES) # this is a helper function for creating
                                                                            # each individual
    toolbox.register("individual", tools.initIterate, creator.Individual,   # this is the function for creating an 
                     toolbox.indices)                                       # individual


    #toolbox.individual()  # a test 

    # implement function for creating a population
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, n = POP_SIZE)

    #toolbox.population()  # a test
    
    def sub_rf_predict(sub_rf, X_test):
        """
        return the predict result using the sub_rf and X_test data;
        the rule is that predict result(labels) with the maximum number of votes wins.
        """
        predict_results = []
        for tree in sub_rf:
            prediction = tree.predict(X_test)
            # record prediction result for a tree
            predict_results.append(prediction)

        # compute the vote_result, i.e. the final result
        y_predict = [0]*len(X_test)
        for idx in range(len(X_test)):
            # for each test data
            # create a vote result
            v_result = vote_result()
            for predict_tree in predict_results:
                v_result[predict_tree[idx]] += 1

            # final result
            y_predict[idx] = keywithmaxval(v_result)

        return  np.array(y_predict, dtype = float)

    # helper function
    y_set = set(y_test).union(y_train)
    def vote_result():
        result = {}
        for k in y_set:
            result[k] = 0
        return result

    def keywithmaxval(d):
        """ a) create a list of the dict's keys and values; 
        b) return the key with the max value"""  
        v=list(d.values())
        k=list(d.keys())
        return k[v.index(max(v))]
    
    
    def evaluate(individual):
        # return the accuracy on the test data
        sub_random_forest = []
        for tree_idx in individual[0: IND_SIZE]:
            sub_random_forest.append(estimators[tree_idx])

        predict_sub_trees = sub_rf_predict(sub_random_forest, X_test)
        # print(predict_sub_trees.__repr__())
        # score = precision_score(y_test, predict_sub_trees, average = 'macro')
        cf_matrix = evaluate_confusion_matrix(individual)
        
        pseudo_acc_case_rate = cf_matrix[1, 1] / np.sum(cf_matrix)
        bad_case_rate = (cf_matrix[1, 0] + cf_matrix[1, 2]) / np.sum(cf_matrix)
        undesired_case_rate = (cf_matrix[0, 1] + cf_matrix[2, 1]) / np.sum(cf_matrix)
        
        # give accuracy more weight
        score = (0.4 * pseudo_acc_case_rate - 0.4 * bad_case_rate - 0.2 * undesired_case_rate) # the max score is 1 and the min score is 0.
        return  score,  # must return an tuple!!!!
    
    def evaluate_confusion_matrix(individual):
        # return the confusion matrix of a model
        sub_random_forest = []
        for tree_idx in individual[0: IND_SIZE]:
            sub_random_forest.append(estimators[tree_idx])  

        predict_sub_trees = sub_rf_predict(sub_random_forest, X_test)
        cf_matrix = confusion_matrix(y_test, predict_sub_trees)
        return  cf_matrix 

    
    
    # implement mutation operator
    mutation_op = tools.mutShuffleIndexes
    
    
    # implement crossover
    def crossover_op(ind1, ind2):
        # only cross over the first IND_SIZE elements in the individual in place
        crossover_idx = random.randint(0, IND_SIZE - 2)
        # print(crossover_idx)
        temp = toolbox.clone(ind1[crossover_idx + 1: IND_SIZE])
        ind1[crossover_idx + 1: IND_SIZE] = ind2[crossover_idx + 1: IND_SIZE]
        ind2[crossover_idx + 1: IND_SIZE] = temp
        return (ind1, ind2)
    
    # implement selection operator
    selection_op = tools.selTournament
    
    
    # register everything in our toolbox
    toolbox.register("mate", crossover_op)
    toolbox.register("mutate", mutation_op, indpb = MUTATE_RATE)
    toolbox.register("select", selection_op, tournsize=3)
    toolbox.register("evaluate", evaluate)
    
    
    h_fame = tools.HallOfFame(100) # keep track of the first 100 best individuals and store them in h_fame

    pop = toolbox.population()
    final_pop = algorithms.eaSimple(pop, toolbox, cxpb = CX_RATE, mutpb=MUTATE_RATE, ngen=1000, 
                                    stats = None, halloffame = h_fame, verbose = False)
    
    # accuracy_of_the_best_individual = evaluate(h_fame[0])
    # accuracy_of_the_whole_trees_model = accuracy_score(y_test, model.predict(X_test))
    cf_matrix_RF_model = confusion_matrix(y_test, model.predict(X_test))
    cf_matrix_GA_RF_model = evaluate_confusion_matrix(h_fame[0])
    
    return cf_matrix_GA_RF_model, cf_matrix_RF_model, evaluate_confusion_matrix, h_fame, estimators




In [35]:
def measure_model_performance(C):
    """
    C is a confusion matrix
    accuracy, sensitivity, specificity: the higher the better
    FP_rate: the lower the better
    
    Now, only compute accuracy
    """
    accuracy = (C[0,0] + C[1, 1] + C[2, 2]) / np.sum(C)
    sensitivity = 0
    specificity = 0 
    FP_rate = 0
    
    cf_matrix = C
    pseudo_acc_case_rate = cf_matrix[1, 1] / np.sum(cf_matrix)
    bad_case_rate = (cf_matrix[1, 0] + cf_matrix[1, 2]) / np.sum(cf_matrix)
    undesired_case_rate = (cf_matrix[0, 1] + cf_matrix[2, 1]) / np.sum(cf_matrix)
    
    left_diag_case_rate = (cf_matrix[0, 0] + cf_matrix[2, 2]) / np.sum(cf_matrix)
    right_diag_case_rate = (cf_matrix[0, 2] + cf_matrix[2, 0]) / np.sum(cf_matrix)
        # give accuracy more weight
    score = (0.4 * pseudo_acc_case_rate - 0.4 * bad_case_rate - 0.2 * undesired_case_rate)
    
    return accuracy, sensitivity, specificity, FP_rate, score

In [36]:
import warnings; warnings.simplefilter('ignore')
num_trees = 100
GA_accuracy_result = []
RF_accuracy_result = []

GA_sensitivity_result = []
RF_sensitivity_result = []

GA_specificity_result = []
RF_specificity_result = []

GA_FP_rate_result = []
RF_FP_rate_result = []

confusion_matrices_list_GA_RF = []
h_fame_list = []
GA_RF_model_list = []

GA_score_list = []
RF_score_list = []

print('num_trees = ', num_trees)
for i in range(100):  # run the experiment 300 times
    print(i)
    cf_matrix_GA_RF_model, cf_matrix_RF_model, evaluate_confusion_matrix, h_fame, estimators = run_GA(num_trees, 10, 30)
    
    GA_accuracy, GA_sensitivity, GA_specificity, GA_FP_rate, GA_score = measure_model_performance(cf_matrix_GA_RF_model)
    RF_accuracy, RF_sensitivity, RF_specificity, RF_FP_rate, RF_score = measure_model_performance(cf_matrix_RF_model)

    GA_accuracy_result.append(GA_accuracy)
    RF_accuracy_result.append(RF_accuracy)

    GA_sensitivity_result.append(GA_sensitivity)
    RF_sensitivity_result.append(RF_sensitivity)

    GA_specificity_result.append(GA_specificity)
    RF_specificity_result.append(RF_specificity)

    GA_FP_rate_result.append(GA_FP_rate)
    RF_FP_rate_result.append(RF_FP_rate)
    
    confusion_matrices_list_GA_RF.append(cf_matrix_GA_RF_model)
    GA_RF_model_list.append(estimators)
    
    GA_score_list.append(GA_score)
    RF_score_list.append(RF_score)

num_trees =  100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [37]:
def show_result():
    print("mean accuracy = ",  np.mean(GA_accuracy_result))
    print("std accuracy = ", np.std(GA_accuracy_result))
    print("Max accuracy = ", np.max(GA_accuracy_result))
    print("confusion Matrix for model with max accuracy is \n", confusion_matrices_list_GA_RF[np.argmax(GA_accuracy_result)])
    print("Max score = ", np.max(GA_score_list))
    print("confusion Matrix for model with max score is \n", confusion_matrices_list_GA_RF[np.argmax(GA_score_list)])

In [38]:
show_result()

mean accuracy =  0.6044
std accuracy =  0.04490206627267341
Max accuracy =  0.72
confusion Matrix for model with max accuracy is 
 [[18  2  5]
 [ 4 19  2]
 [ 5  3 17]]
Max score =  0.10933333333333334
confusion Matrix for model with max score is 
 [[14  5  6]
 [ 0 25  0]
 [ 9  4 12]]


In [57]:
max_score = 2/15
min_score = -1/5
def normalize_score(scores, max_score, min_score):
    np_scores = np.array(scores)
    return (np_scores - min_score)/(max_score - min_score)

In [60]:
GA_norm_score = normalize_score(GA_score_list, max_score, min_score)
GA_norm_score

array([0.768, 0.808, 0.768, 0.768, 0.744, 0.8  , 0.704, 0.728, 0.808,
       0.84 , 0.792, 0.832, 0.696, 0.928, 0.776, 0.704, 0.768, 0.776,
       0.792, 0.856, 0.816, 0.68 , 0.792, 0.832, 0.848, 0.8  , 0.816,
       0.808, 0.792, 0.736, 0.68 , 0.8  , 0.736, 0.72 , 0.848, 0.776,
       0.728, 0.776, 0.784, 0.712, 0.768, 0.768, 0.696, 0.816, 0.792,
       0.808, 0.736, 0.84 , 0.728, 0.856, 0.792, 0.704, 0.792, 0.744,
       0.848, 0.832, 0.792, 0.808, 0.736, 0.784, 0.72 , 0.672, 0.776,
       0.864, 0.672, 0.792, 0.856, 0.832, 0.728, 0.88 , 0.712, 0.848,
       0.808, 0.848, 0.776, 0.784, 0.824, 0.768, 0.864, 0.68 , 0.792,
       0.8  , 0.768, 0.8  , 0.8  , 0.76 , 0.808, 0.784, 0.784, 0.792,
       0.76 , 0.856, 0.8  , 0.792, 0.768, 0.848, 0.856, 0.784, 0.8  ,
       0.768])

In [62]:
RF_norm_score = normalize_score(RF_score_list, max_score, min_score)
RF_norm_score

array([0.416, 0.464, 0.408, 0.24 , 0.496, 0.52 , 0.256, 0.392, 0.568,
       0.448, 0.472, 0.544, 0.392, 0.544, 0.4  , 0.424, 0.408, 0.352,
       0.368, 0.392, 0.328, 0.312, 0.376, 0.496, 0.312, 0.432, 0.36 ,
       0.264, 0.44 , 0.424, 0.44 , 0.504, 0.416, 0.296, 0.472, 0.368,
       0.264, 0.464, 0.384, 0.4  , 0.328, 0.32 , 0.288, 0.448, 0.512,
       0.296, 0.416, 0.472, 0.416, 0.408, 0.304, 0.392, 0.488, 0.352,
       0.496, 0.376, 0.32 , 0.408, 0.448, 0.424, 0.336, 0.224, 0.52 ,
       0.456, 0.352, 0.32 , 0.384, 0.44 , 0.408, 0.32 , 0.304, 0.456,
       0.464, 0.592, 0.328, 0.504, 0.568, 0.392, 0.448, 0.344, 0.44 ,
       0.472, 0.416, 0.552, 0.496, 0.392, 0.384, 0.32 , 0.424, 0.24 ,
       0.352, 0.336, 0.432, 0.488, 0.416, 0.48 , 0.528, 0.424, 0.408,
       0.536])